### 1. Importing Libraries

Firstly,  I load in the libraries I will need the below codes.

In [219]:
# Importing necessary libraries
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from keras import backend as K
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import backend as k
import matplotlib.pyplot as plt
import numpy as np

### 2. Reading  Train Datas in Images

The fruit 360 dataset contains pre defined training and test splits, divided into two folders. Instead of using these, I am going to read in fruit images from the training folders, and create my own train/test splits later.

In [220]:
training_datas = list()
training_labels = list()

In [221]:
for dirPath in glob.glob("fruits/fruits-360/Training/*"):
    imageLabel = dirPath.split('/')[-1]
    for imagePath in glob.glob(os.path.join(dirPath, '*')):
        image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (32, 32))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        training_datas.append(image)
        training_labels.append(imageLabel)    

In [222]:
len(training_datas), len(training_labels)

(60486, 60486)

In [223]:
training_datas = np.array(training_datas)
training_labels = np.array(training_labels)

In [224]:
training_datas.shape

(60486, 32, 32, 3)

In [225]:
id_to_label = {key : value for key, value in enumerate(np.unique(training_labels))}
label_to_id = {key:value for value, key in id_to_label.items()} 

In [226]:
training_labels = np.array([label_to_id[item] for item in training_labels])

In [227]:
print("training data shape = {1}, training labels shape =  {0}".format(training_labels.shape, training_datas.shape))

training data shape = (60486, 32, 32, 3), training labels shape =  (60486,)


In [235]:
fname = list(id_to_label.values())

### 3. Reading Validation Datas in Images

In [236]:
validation_datas = list()
validation_labels = list()

In [237]:
for dirPath in glob.glob("fruits/fruits-360/Test/*"):
    imageLabel = dirPath.split('/')[-1]
    for imagePath in glob.glob(os.path.join(dirPath, '*')):
        image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (32, 32))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        validation_datas.append(image)
        validation_labels.append(imageLabel)    

In [238]:
len(validation_datas), len(validation_labels)

(20618, 20618)

In [239]:
validation_datas = np.array(validation_datas)
validation_labels = np.array(validation_labels)

In [240]:
validation_labels = np.array([label_to_id[item] for item in validation_labels])

In [241]:
print("validation data shape = {1}, validation labels shape =  {0}".format(validation_labels.shape, validation_datas.shape))

validation data shape = (20618, 32, 32, 3), validation labels shape =  (20618,)


### 4. Preparing the data for training

In [242]:
training_set_x_n, training_set_y_n = training_datas,  training_labels
validation_set_x_n, validation_set_y_n = validation_datas,  validation_labels

In [243]:
#Normalization of the images pixels.
training_set_x_n = training_set_x_n / 255
validation_set_x_n = validation_set_x_n / 255

In [244]:
flat_training_set_x_n = training_set_x_n.reshape(training_set_x_n.shape[0], 32*32*3)
flat_validation_set_x_n = validation_set_x_n.reshape(validation_set_x_n.shape[0], 32*32*3)

In [245]:
np.unique(training_set_y_n).shape

(120,)

In [246]:
validation_set_y_n

array([115, 115, 115, ...,  77,  77,  77])

In [247]:
#One Hot Encode the Output
training_set_y_n = keras.utils.to_categorical(training_set_y_n)
validation_set_y_n = keras.utils.to_categorical(validation_set_y_n)

In [248]:
len(id_to_label)

120

In [249]:
training_set_y_n.shape

(60486, 120)

In [250]:
print('Original Sizes:', training_set_x_n.shape, validation_set_x_n.shape, training_set_y_n.shape, validation_set_y_n.shape)
print('Flattened:', flat_training_set_x_n.shape, flat_training_set_x_n.shape)

Original Sizes: (60486, 32, 32, 3) (20618, 32, 32, 3) (60486, 120) (20618, 120)
Flattened: (60486, 3072) (60486, 3072)


### 5. TRAINING ...

In [251]:
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,Activation,BatchNormalization
from keras.optimizers import Adamax
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K

In [254]:
model = Sequential(name = 'Fruit Classification')
model.add(Conv2D(16, (3, 3), input_shape=(32,32,3),padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(32, (3,3), padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(LeakyReLU(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
#model.add(LeakyReLU(0.1))
model.add(Dropout(0.5))
model.add(Dense(120))
model.add(Activation("softmax"))

model.summary()

Model: "Fruit Classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 32, 32, 16)        64        
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 16, 16, 32

In [253]:
model.compile(loss='categorical_crossentropy',
             optimizer = Adamax(),
             metrics=['accuracy'])

model.fit(training_set_x_n,
          training_set_y_n,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data = (validation_set_x_n, validation_set_y_n)
         )

Train on 60486 samples, validate on 20618 samples
Epoch 1/10
60486/60486 [==============================] - 68s 1ms/step - loss: 1.2607 - accuracy: 0.6674 - val_loss: 3.5854 - val_accuracy: 0.2343
Epoch 2/10
60486/60486 [==============================] - 70s 1ms/step - loss: 0.2206 - accuracy: 0.9385 - val_loss: 0.1524 - val_accuracy: 0.9579
Epoch 3/10
60486/60486 [==============================] - 71s 1ms/step - loss: 0.0995 - accuracy: 0.9721 - val_loss: 0.1312 - val_accuracy: 0.9575
Epoch 4/10
60486/60486 [==============================] - 68s 1ms/step - loss: 0.0597 - accuracy: 0.9835 - val_loss: 0.1066 - val_accuracy: 0.9687
Epoch 5/10
60486/60486 [==============================] - 65s 1ms/step - loss: 0.0378 - accuracy: 0.9904 - val_loss: 0.0614 - val_accuracy: 0.9824
Epoch 6/10
60486/60486 [==============================] - 67s 1ms/step - loss: 0.0280 - accuracy: 0.9929 - val_loss: 0.0605 - val_accuracy: 0.9821
Epoch 7/10
60486/60486 [==============================] - 69s 1ms/st